# Calcoli Strutturali - Stampante 3D Custom 900mm

Notebook interattivo per dimensionamento e verifica strutturale della stampante 3D.

**Specifiche progetto:**
- Volume di stampa: 900×740×800mm
- Profili acciaio: 40×20×2mm
- Materiale: Acciaio S235 (E = 210 GPa)

In [ ]:
# Import librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Configurazione grafici
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Calcolo Momento di Inerzia Profilo

Profilo rettangolare cavo 40×20×2mm

In [ ]:
# Dimensioni profilo [mm]
b_ext = 40  # Base esterna
h_ext = 20  # Altezza esterna
t = 2       # Spessore parete

b_int = b_ext - 2*t  # Base interna
h_int = h_ext - 2*t  # Altezza interna

# Momento di inerzia rispetto asse forte (flessione verticale)
I_forte = (b_ext * h_ext**3 - b_int * h_int**3) / 12

# Momento di inerzia rispetto asse debole (flessione laterale)
I_debole = (h_ext * b_ext**3 - h_int * b_int**3) / 12

# Area sezione
A = b_ext * h_ext - b_int * h_int

print(f"📐 PROFILO: {b_ext}×{h_ext}×{t} mm")
print(f"   Area sezione: {A:.1f} mm²")
print(f"   Momento inerzia asse forte: {I_forte:.1f} mm⁴")
print(f"   Momento inerzia asse debole: {I_debole:.1f} mm⁴")
print(f"   Rapporto I_forte/I_debole: {I_forte/I_debole:.2f}x")

## 2. Calcolo Flessione Trave

Formula: $ \delta = \frac{F \cdot L^3}{48 \cdot E \cdot I} $ (trave appoggiata, carico centrato)

In [ ]:
# Parametri calcolo
L = 900          # Lunghezza trave [mm]
F = 50           # Forza applicata [N]
E = 210000       # Modulo elasticità acciaio [N/mm²]
I = I_debole     # Usiamo momento inerzia asse debole (caso critico)

# Calcolo flessione
flessione = (F * L**3) / (48 * E * I)

print(f"⚙️ FLESSIONE TRAVE")
print(f"   Lunghezza: {L} mm")
print(f"   Carico: {F} N")
print(f"   Flessione massima: {flessione:.2f} mm")
print(f"   Rapporto flessione/lunghezza: 1/{L/flessione:.0f}")

# Verifica tolleranza (regola empirica: flessione < L/500)
flessione_limite = L / 500
if flessione < flessione_limite:
    print(f"   ✅ OK: Flessione entro limiti ({flessione_limite:.2f} mm)")
else:
    print(f"   ⚠️ ATTENZIONE: Flessione eccessiva (limite: {flessione_limite:.2f} mm)")

## 3. Analisi Parametrica: Flessione vs Carico

In [ ]:
# Range di carichi da analizzare
carichi = np.linspace(10, 200, 50)  # [N]
flessioni = (carichi * L**3) / (48 * E * I)

# Grafico
plt.figure(figsize=(10, 6))
plt.plot(carichi, flessioni, 'b-', linewidth=2, label='Flessione calcolata')
plt.axhline(y=flessione_limite, color='r', linestyle='--', label=f'Limite L/500 = {flessione_limite:.2f} mm')
plt.axvline(x=F, color='g', linestyle=':', alpha=0.7, label=f'Carico progetto = {F} N')

plt.xlabel('Carico [N]', fontsize=12)
plt.ylabel('Flessione [mm]', fontsize=12)
plt.title(f'Analisi Flessione - Trave {L}mm (Profilo {b_ext}×{h_ext}×{t})', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

# Calcolo carico massimo ammissibile
F_max = (flessione_limite * 48 * E * I) / L**3
print(f"\n📊 Carico massimo ammissibile (flessione < L/500): {F_max:.1f} N")
print(f"   Coefficiente sicurezza attuale: {F_max/F:.2f}x")

## 4. Analisi Frequenze di Risonanza

Frequenza naturale trave: $ f = \frac{\lambda^2}{2\pi L^2} \sqrt{\frac{EI}{\mu}} $

dove λ = 3.927 (primo modo trave appoggiata)

In [ ]:
# Densità acciaio
rho = 7850e-9  # [kg/mm³]
mu = A * rho   # Massa per unità di lunghezza [kg/mm]

# Coefficiente primo modo (trave appoggiata)
lambda_1 = 3.927

# Frequenza naturale [Hz]
f_naturale = (lambda_1**2 / (2 * np.pi * L**2)) * np.sqrt((E * I) / mu)

print(f"🎵 ANALISI FREQUENZE")
print(f"   Massa lineare: {mu*1000:.3f} g/mm = {mu*L:.2f} kg/trave")
print(f"   Frequenza naturale (1° modo): {f_naturale:.2f} Hz")

# Frequenze critiche stampanti 3D
freq_stepper = 200  # Steps/rev tipico motore
rpm_max = 3000      # RPM massimi tipici
freq_operativa = (freq_stepper * rpm_max) / 60

print(f"   Frequenza operativa motori: {freq_operativa:.0f} Hz")

if f_naturale > freq_operativa * 2:
    print(f"   ✅ OK: Frequenza naturale ben separata da frequenze operative")
else:
    print(f"   ⚠️ ATTENZIONE: Rischio risonanza con frequenze operative")

## 5. Stima Carichi Reali Stampante

Calcolo dei carichi effettivi agenti sulla struttura

In [ ]:
# Masse componenti [kg]
m_piatto = 15.0       # Piatto + supporto
m_estrusore = 2.0     # Hotend + extruder
m_guide = 8.0         # Guide lineari
m_motori = 4.0        # Motori stepper
m_elettronica = 3.0   # Scheda + alimentatori

# Peso totale mobile sull'asse X (caso più critico)
m_totale_X = m_piatto
P_X = m_totale_X * 9.81  # Peso [N]

# Peso totale mobile sull'asse Y
m_totale_Y = m_estrusore
P_Y = m_totale_Y * 9.81  # Peso [N]

# Accelerazioni stampante
a_max = 3000  # mm/s² (tipico per grandi stampanti)
F_inerzia_X = (m_totale_X * a_max) / 1000  # Forza inerziale [N]
F_inerzia_Y = (m_totale_Y * a_max) / 1000  # Forza inerziale [N]

print(f"⚖️ CARICHI REALI")
print(f"\n   Asse X (piatto mobile):")
print(f"   - Massa: {m_totale_X:.1f} kg")
print(f"   - Peso statico: {P_X:.1f} N")
print(f"   - Forza inerziale (acc {a_max} mm/s²): {F_inerzia_X:.1f} N")
print(f"   - Carico totale equivalente: {P_X + F_inerzia_X:.1f} N")

print(f"\n   Asse Y (estrusore):")
print(f"   - Massa: {m_totale_Y:.1f} kg")
print(f"   - Peso statico: {P_Y:.1f} N")
print(f"   - Forza inerziale: {F_inerzia_Y:.1f} N")
print(f"   - Carico totale equivalente: {P_Y + F_inerzia_Y:.1f} N")

# Verifica flessione con carichi reali
flessione_reale_X = ((P_X + F_inerzia_X) * L**3) / (48 * E * I)
print(f"\n   Flessione asse X: {flessione_reale_X:.2f} mm")
if flessione_reale_X < flessione_limite:
    print(f"   ✅ OK: Entro limiti ({flessione_limite:.2f} mm)")
else:
    print(f"   ⚠️ ATTENZIONE: Eccessiva (limite: {flessione_limite:.2f} mm)")

## 6. Confronto Profili Alternativi

Analisi comparativa con altri profili standard

In [ ]:
# Database profili [b, h, t] mm
profili = {
    '30×20×2': (30, 20, 2),
    '40×20×2': (40, 20, 2),  # Profilo attuale
    '40×40×2': (40, 40, 2),
    '50×30×2': (50, 30, 2),
    '60×40×3': (60, 40, 3),
}

risultati = []

for nome, (b, h, s) in profili.items():
    # Calcolo proprietà
    b_i = b - 2*s
    h_i = h - 2*s
    I_calc = (h * b**3 - h_i * b_i**3) / 12
    A_calc = b * h - b_i * h_i
    
    # Flessione
    flex = (P_X * L**3) / (48 * E * I_calc)
    
    # Massa
    massa = A_calc * L * rho * 1000  # [g]
    
    risultati.append({
        'Profilo': nome,
        'Area [mm²]': A_calc,
        'I [mm⁴]': I_calc,
        'Flessione [mm]': flex,
        'Massa [g]': massa,
        'OK': '✅' if flex < flessione_limite else '❌'
    })

# Visualizzazione risultati
import pandas as pd
df = pd.DataFrame(risultati)
df = df.set_index('Profilo')
print("\n📋 CONFRONTO PROFILI\n")
print(df.to_string())

# Grafico confronto
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Flessione
ax1.bar(df.index, df['Flessione [mm]'], color=['green' if x == '✅' else 'red' for x in df['OK']])
ax1.axhline(y=flessione_limite, color='orange', linestyle='--', label=f'Limite = {flessione_limite:.2f} mm')
ax1.set_ylabel('Flessione [mm]', fontsize=11)
ax1.set_title('Flessione per Profilo', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Massa
colors = ['blue' if '40×20×2' in x else 'gray' for x in df.index]
ax2.bar(df.index, df['Massa [g]'], color=colors, alpha=0.7)
ax2.set_ylabel('Massa [g]', fontsize=11)
ax2.set_title('Massa Trave (L=900mm)', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Riepilogo e Conclusioni

In [ ]:
print("="*60)
print("   📊 RIEPILOGO CALCOLI STRUTTURALI")
print("="*60)
print(f"\n🔧 Configurazione Scelta:")
print(f"   - Profilo: {b_ext}×{h_ext}×{t} mm")
print(f"   - Lunghezza trave critica: {L} mm")
print(f"   - Momento inerzia: {I:.1f} mm⁴")

print(f"\n⚖️ Carichi:")
print(f"   - Peso statico: {P_X:.1f} N")
print(f"   - Forza inerziale: {F_inerzia_X:.1f} N")
print(f"   - Totale: {P_X + F_inerzia_X:.1f} N")

print(f"\n📏 Flessione:")
print(f"   - Calcolata: {flessione_reale_X:.2f} mm")
print(f"   - Limite ammissibile: {flessione_limite:.2f} mm")
print(f"   - Rapporto: {flessione_reale_X/flessione_limite:.1%}")
print(f"   - Status: {'✅ OK' if flessione_reale_X < flessione_limite else '⚠️ CRITICO'}")

print(f"\n🎵 Dinamica:")
print(f"   - Frequenza naturale: {f_naturale:.1f} Hz")
print(f"   - Status: {'✅ OK' if f_naturale > freq_operativa * 2 else '⚠️ CONTROLLARE'}")

print(f"\n💰 Stima Massa Struttura:")
travi_totali = 12  # Stima per struttura completa
massa_totale = (A * L * rho * 1000 * travi_totali) / 1000  # [kg]
print(f"   - Massa singola trave: {(A * L * rho * 1000):.1f} g")
print(f"   - Massa struttura (~{travi_totali} travi): {massa_totale:.1f} kg")

print("\n" + "="*60)
print("   ✅ Analisi completata")
print("="*60)

---

## 📝 Note Tecniche

**Formule utilizzate:**
- Flessione trave appoggiata con carico concentrato al centro: δ = FL³/(48EI)
- Momento di inerzia profilo rettangolare cavo: I = (bh³ - b'h'³)/12
- Frequenza naturale trave: f = (λ²/2πL²)√(EI/μ)

**Ipotesi:**
- Travi appoggiate agli estremi (vincoli semplici)
- Materiale elastico lineare (acciaio S235)
- Carico statico equivalente (peso + inerzia)
- Deformazioni piccole (teoria trave di Eulero-Bernoulli)

**Limiti di validità:**
- Flessione < L/500 (rigidità stampanti 3D)
- Stress < σ_snervamento (non calcolato, assume design conservativo)
- Frequenze operative < f_naturale/2 (evita risonanza)

---

*Notebook creato per: Stampante 3D Custom 900×740×800mm*  
*Autore: [Il tuo nome]*  
*Data: 2026-01-25*